In [1]:
#—————————————————————————————————————————————————— INPUT BOX ——————————————————————————————————————————#
# Input query to search and extracting papers 输入值

# key word for searching 关键词搜素
query = "Antimony Selenide solar cell"

# saved path 需要保存的文件路径
path  = '/Users/yitengxu/Desktop/Project/Final Handover Code/Paper Scrapers and Parser/RSCPapers'


In [47]:
# -*- coding: utf-8 -*-
"""
Royal Society of Chemistry web-scraper. Please get the permission from RSC before web-scraping.
"""
import os
import re
import requests
import urllib.request
from urllib.parse import quote
from chemdataextractor.scrape.pub.rsc import RscSearchScraper
from selenium import webdriver





# This doi dependent in certain webpage   只统计网页里一页的doi 有25个
def get_doi(query, pagenumber, output_file):
    """
    Find the DOIs of a topic in RSC
    :param query: <str> search topic
    :param pagenumber: <int> the page of the search items
    :param output_file: <str> output file path
    :return: <list> of <str> list of DOIs
    """
    # Create a Chrome WebDriver instance
    driver = webdriver.Chrome(executable_path='/Users/yitengxu/Desktop/PaperScraper-master/paperscraper/webdrivers/chromedriver_mac64/chromedriver')

    try:
        scrape = RscSearchScraper(driver=driver).run(query, page=pagenumber)
        results = scrape.serialize()
        dois = [result['doi'] for result in results]

        filename = open(f'{path}/DOI_RSC_PV', 'a+')
        for i in dois:
            filename.write(i+'\n')
        filename.close()

        return dois
    finally:
        # Quit the driver to close the browser window and release resources
        driver.quit()
    


# def way to find number of result.  P.S: each page include 25 result.  统计搜索后多少结果数
def extract_result_count(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    response = requests.get(url, headers=headers)
    text = response.text

    result = re.search(r'"Count":(\d+)', text)
    if result:
        count = int(result.group(1))
        return count
    else:
        return print('None found')

# 统计所以结果的doi ，最后doi的运行指令
def get_total_doi(result_count):
    import math
    
    numberpages = math.ceil(result_count / 25)
    
    for i in range(1, numberpages + 1):
        get_doi(query, i, path)


def get_url(doi):
    """
    Find the full url address according to the DOI
    :param doi: <str> DOI
    :return: <str> url text
    """
    try:
        r = requests.get(
            'http://doi.org/' +
            doi,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})
        result = re.findall(
            r'https://pubs.rsc.org/en/content/articlehtml/.*?"',
            r.text)
        result = result[0][:-1]
        return result
    except IndexError:
        return None


def download_doi(dois):
    """
    Download the paper according to the DOI
    :param dois: <list> list of DOIs
    """
    count = 0
    for i in dois:
        try:
            count += 1
            webcontent = urllib.request.urlopen(get_url(i)).read()
            f = open(f"{path}/RSCHtml/{count}.html", 'wb')
            f.write(webcontent)
            f.close()
        except AttributeError:
            continue


In [4]:
# clearing any file runned  清除已经运行过的doi文件残留


filepath = f'{path}/DOI_RSC_PV' # ！这里有文件命名
if os.path.isfile(filepath):
    os.remove(filepath)

In [48]:
# To find number of result in this query 


url = "http://pubs.rsc.org/en/results?searchtext=" 

encoded_query = quote(query)
url_searched =url + encoded_query

result_count = extract_result_count(url_searched)
print("Number of papers results:", result_count)


Number of papers results: 216


In [6]:
get_total_doi(result_count)

In [49]:
# 在DOI文件里清除重复的DOI 元素
# To prevent repeated DIO string, file will remove extra one  

filename = f'{path}/DOI_RSC_PV' # ！文件命名
with open(filename, 'r+') as file:
    lines = file.readlines()
    file.seek(0)  # 将文件指针移回文件开头
    unique_lines = set(lines)  # 使用集合来去除重复的行
    file.writelines(unique_lines)
    file.truncate()  # 截断文件，删除多余的内容

file1 = open(f'{path}/DOI_RSC_PV', 'r')  # ！文件命名
Lines = file1.readlines()
count = 0
# Strips the newline character
for line in Lines:
    count += 1
print('There are ' + str(count) + ' DOI lists in DOI File')

There are 216 DOI lists in DOI File


In [50]:
# Read dois in a list ,读取写入的dois 内容作为list

def read_doi_file(file_path):
    with open(file_path, 'r') as file:
        dois = [line.strip() for line in file]
    return dois

dois = read_doi_file(filename)





In [51]:
# Final html file paper output   最后的html格式论文输出
download_doi(dois)

In [ ]:
# ！注意这里输出结果数字不一定是连续的，未出现的数字意味着该doi元素无html格式资源